In [42]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re
from sqlalchemy import create_engine
from urllib.parse import quote_plus
from dotenv import load_dotenv
import os
import json

In [43]:
load_dotenv()
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=os.getenv("client_id"),client_secret=os.getenv("client_secret")
))

In [55]:
playlist_url = 'https://open.spotify.com/playlist/4ChdbsG6VLRzrpsPku66Tf'
match = re.search(r'playlist/([a-zA-Z0-9]+)', playlist_url)

if match:
    playlist_id = match.group(1)
    print("Track ID:", playlist_id)
else:
    print("No playlist ID found in URL")


Track ID: 4ChdbsG6VLRzrpsPku66Tf


In [56]:
playlist = sp.playlist(playlist_id)

playlist_name = playlist['name']
playlist_owner = playlist['owner']['display_name']
follower_count = playlist['followers']['total']

print(f"Playlist Name: {playlist_name}")
print(f"Owner: {playlist_owner}")
print(f"Followers: {follower_count}")

Playlist Name: Sounds..🎧
Owner: Mugu Maddy
Followers: 1


In [57]:
with open("playlist_data.json", "w") as f:
    json.dump(playlist, f, indent=4)

In [58]:
tracks = []
offset = 0

while True:
    response = sp.playlist_tracks(playlist_id, offset=offset, limit=100)
    items = response['items']
    
    if not items:
        break  # No more songs

    for item in items:
        track = item['track']
        tracks.append({
            'track_name': track['name'],
            'album': track['album']['name'],
            'artist': track['artists'][0]['name'],
            'popularity': track['popularity'],
            'duration_min': round(track['duration_ms'] / 60000,2)
        })

    offset += 100  # move to next batch

df = pd.DataFrame(tracks)
df.sample(5)

,track_name,album,artist,popularity,duration_min
89,Bom Bidi Bom,Fifty Shades Darker (Original Motion Picture S...,Nick Jonas,55,3.58
160,MY EYES,UTOPIA,Travis Scott,81,4.19
62,Circles,Hollywood's Bleeding,Post Malone,86,3.59
88,Summertime Sadness,Born To Die - The Paradise Edition,Lana Del Rey,84,4.42
194,Dancin - Krono Remix,Ibiza 2019,Aaron Smith,73,4.27


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   track_name    210 non-null    object 
 1   album         210 non-null    object 
 2   artist        210 non-null    object 
 3   popularity    210 non-null    int64  
 4   duration_min  210 non-null    float64
dtypes: float64(1), int64(1), object(3)
memory usage: 8.3+ KB


In [61]:
df.to_csv('spotify_sounds_data.csv', index=False)

In [62]:
load_dotenv()

username = os.getenv("db_user")
password = os.getenv("db_password")
host = os.getenv("db_host")
database = os.getenv("db_name")

engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{database}")
try:
    with engine.connect() as connection:
        print("Connection successful!")
except Exception as e:
    print("Connection failed:")
    print(e)

Connection successful!


In [63]:
df.to_sql(name='sounds_eng', con=engine,if_exists='replace',index=False, chunksize=1000)

210

In [64]:
pd.read_sql('select * from sounds_eng;',engine)

,track_name,album,artist,popularity,duration_min
0,goosebumps,Birds In The Trap Sing McKnight,Travis Scott,85,4.06
1,Pray,Fifty Shades Darker (Original Motion Picture S...,JRY,54,3.35
2,Death Grip,Death Grip,Watt White,0,2.96
3,Canzoni preferite,JOJO'S BIZARRE ADVENTURE -Golden Wind O.S.T vo...,Yugo Kanno,59,2.41
4,Heat Waves,Dreamland,Glass Animals,86,3.98
...,...,...,...,...,...
205,Beat It,Thriller,Michael Jackson,83,4.30
206,rockstar (feat. 21 Savage),beerbongs & bentleys,Post Malone,82,3.64
207,Wow.,Hollywood's Bleeding,Post Malone,78,2.49
208,Whatever It Takes,Evolve,Imagine Dragons,83,3.35
